In [1]:
# ЯЧЕЙКА 1: Импорты и базовые настройки
import os
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
from tqdm import tqdm

RMT_INFER_DIR = "/mnt/10tb/home/patsukevichan/borZoi/borzoi/examples/rmt_infer/"
BORZOI_BEDGRAPH_DIR = "/mnt/10tb/home/patsukevichan/borZoi/borzoi/examples/predicted_expression_by_chromosomes/"

print("Libraries imported. Paths set.")


Libraries imported. Paths set.


In [2]:
# ЯЧЕЙКА 2: Загрузка базовых parquet-файлов (labels, transcripts, RMT-предикты)

labels_parquet = os.path.join(RMT_INFER_DIR, "labels.parquet")
labels_transcripts_parquet = os.path.join(RMT_INFER_DIR, "labels_transcripts.parquet")
preds_2seg_parquet = os.path.join(RMT_INFER_DIR, "preds_transcripts_2seg.parquet")
preds_4seg_parquet = os.path.join(RMT_INFER_DIR, "preds_transcripts_4seg.parquet")
preds_8seg_parquet = os.path.join(RMT_INFER_DIR, "preds_transcripts_8seg.parquet")

labels_df = pd.read_parquet(labels_parquet)
labels_transcripts_df = pd.read_parquet(labels_transcripts_parquet)
preds_2seg_df = pd.read_parquet(preds_2seg_parquet)
preds_4seg_df = pd.read_parquet(preds_4seg_parquet)
preds_8seg_df = pd.read_parquet(preds_8seg_parquet)

print("labels_df:")
display(labels_df.head())
print("labels_df[preds].min/max/mean:",
      labels_df["preds"].min(), 
      labels_df["preds"].max(), 
      labels_df["preds"].mean())

print("\nlabels_transcripts_df:")
display(labels_transcripts_df.head())
print("labels_transcripts_df[preds].min/max/mean:",
      labels_transcripts_df["preds"].min(),
      labels_transcripts_df["preds"].max(),
      labels_transcripts_df["preds"].mean())

print("\npreds_2seg_df:")
display(preds_2seg_df.head())
print("preds_2seg_df[preds].min/max/mean:",
      preds_2seg_df["preds"].min(),
      preds_2seg_df["preds"].max(),
      preds_2seg_df["preds"].mean())

print("\npreds_4seg_df:")
display(preds_4seg_df.head())
print("preds_4seg_df[preds].min/max/mean:",
      preds_4seg_df["preds"].min(),
      preds_4seg_df["preds"].max(),
      preds_4seg_df["preds"].mean())

print("\npreds_8seg_df:")
display(preds_8seg_df.head())
print("preds_8seg_df[preds].min/max/mean:",
      preds_8seg_df["preds"].min(),
      preds_8seg_df["preds"].max(),
      preds_8seg_df["preds"].mean())


labels_df:


,chrom_pred,preds,chrom,start,end,start_pred,end_pred,Transcript stable ID,description_id
0,chr10,-4.60517,chr10,72893738,72893859,72893735,72893739,ENST00000334011,ENCFF123KIW
1,chr10,-4.60517,chr10,72893738,72893859,72893739,72893743,ENST00000334011,ENCFF123KIW
2,chr10,-4.60517,chr10,72893738,72893859,72893743,72893749,ENST00000334011,ENCFF123KIW
3,chr10,-4.60517,chr10,72893738,72893859,72893749,72893752,ENST00000334011,ENCFF123KIW
4,chr10,-4.60517,chr10,72893738,72893859,72893752,72893758,ENST00000334011,ENCFF123KIW


labels_df[preds].min/max/mean:
 -4.6051702 13.022035 1.0628820826820662

labels_transcripts_df:


,Transcript stable ID,description_id,preds
0,ENST00000001008,ENCFF123KIW,4.037295
1,ENST00000002596,ENCFF123KIWr,4.281860
2,ENST00000002829,ENCFF123KIW,5.388129
3,ENST00000009180,ENCFF123KIW,7.723541
4,ENST00000009589,ENCFF123KIWr,8.866660


labels_transcripts_df[preds].min/max/mean:
 -4.605170200000001 11.856355566803279 -0.4990853600718094

preds_2seg_df:


...

preds_2seg_df[preds].min/max/mean:
 -3.8326391907692305 6.641538864634146 -1.7986488347158898

preds_4seg_df:


,Transcript stable ID,description_id,preds
0,ENST00000001008,ENCFF123KIW,2.727709
1,ENST00000002596,ENCFF123KIWr,0.732307
2,ENST00000002829,ENCFF123KIW,2.426938
3,ENST00000009180,ENCFF123KIW,4.060136
4,ENST00000009589,ENCFF123KIWr,4.862711


preds_4seg_df[preds].min/max/mean:
 -4.2842083040022745 7.001022550393701 -0.715264122164497

preds_8seg_df:


,Transcript stable ID,description_id,preds
0,ENST00000001008,ENCFF123KIW,3.736615
1,ENST00000002596,ENCFF123KIWr,1.674160
2,ENST00000002829,ENCFF123KIW,0.736134
3,ENST00000009180,ENCFF123KIW,4.146430
4,ENST00000009589,ENCFF123KIWr,5.075033


preds_8seg_df[preds].min/max/mean:
 -4.180466452112676 6.389540544094489 -0.6615956298938233


In [3]:
# ЯЧЕЙКА 3: Формируем "очищенный" DataFrame labels_clear_df
# Убираем лишний столбец preds, оставляем [chrom, start, end, Transcript stable ID, description_id]

labels_clear_df = labels_df[["chrom", "start", "end", "Transcript stable ID", "description_id"]].copy()
labels_clear_df.drop_duplicates(subset=["Transcript stable ID"], inplace=True)

print("labels_clear_df (очищенная):")
display(labels_clear_df.head())
print("Shape:", labels_clear_df.shape)


labels_clear_df (очищенная):


,chrom,start,end,Transcript stable ID,description_id
0,chr10,72893738,72893859,ENST00000334011,ENCFF123KIW
379,chr10,73071294,73071357,ENST00000431293,ENCFF123KIW
425,chr10,73110454,73110567,ENST00000357321,ENCFF123KIW
772,chr10,73168118,73168386,ENST00000242505,ENCFF123KIW
1660,chr10,73247359,73247383,ENST00000440197,ENCFF123KIW


Shape: (4419, 5)


In [4]:
# ЯЧЕЙКА 4: Сопоставление description_id -> имя bedGraph-файла (Borzoi)
# Предположим, что вы уже знаете соответствие.

experiment_mapping = {
    "ENCFF123KIW":  "borzoi_rnaseq_fold4_ENCSR892LBU+_fw.bedGraph",
    "ENCFF123KIWr": "borzoi_rnaseq_fold4_ENCSR892LBU-_fw.bedGraph",
    "ENCFF784MDF":  "borzoi_rnaseq_fold4_ENCSR357BYU+_fw.bedGraph",
    "ENCFF784MDFr": "borzoi_rnaseq_fold4_ENCSR357BYU-_fw.bedGraph",
    "ENCFF236XOK":  "borzoi_rnaseq_fold4_ENCSR763OMY+_fw.bedGraph",
    "ENCFF236XOKr": "borzoi_rnaseq_fold4_ENCSR763OMY-_fw.bedGraph",
    "ENCFF781TTC":  "borzoi_rnaseq_fold4_ENCSR071DYD+_fw.bedGraph",
    "ENCFF781TTCr": "borzoi_rnaseq_fold4_ENCSR071DYD-_fw.bedGraph",
    "ENCFF242BWW":  "borzoi_rnaseq_fold4_ENCSR045GTF+_fw.bedGraph",
    "ENCFF242BWWr": "borzoi_rnaseq_fold4_ENCSR045GTF-_fw.bedGraph"
}

print("experiment_mapping (description_id -> bedGraph file):")
for k,v in experiment_mapping.items():
    print(k, "->", v)


Словарь сопоставлений для description_id:
ENCFF123KIW: borzoi_rnaseq_fold4_ENCSR892LBU+_fw.bedGraph
ENCFF123KIWr: borzoi_rnaseq_fold4_ENCSR892LBU-_fw.bedGraph
ENCFF784MDF: borzoi_rnaseq_fold4_ENCSR357BYU+_fw.bedGraph
ENCFF784MDFr: borzoi_rnaseq_fold4_ENCSR357BYU-_fw.bedGraph
ENCFF236XOK: borzoi_rnaseq_fold4_ENCSR763OMY+_fw.bedGraph
ENCFF236XOKr: borzoi_rnaseq_fold4_ENCSR763OMY-_fw.bedGraph
ENCFF781TTC: borzoi_rnaseq_fold4_ENCSR071DYD+_fw.bedGraph
ENCFF781TTCr: borzoi_rnaseq_fold4_ENCSR071DYD-_fw.bedGraph
ENCFF242BWW: borzoi_rnaseq_fold4_ENCSR045GTF+_fw.bedGraph
ENCFF242BWWr: borzoi_rnaseq_fold4_ENCSR045GTF-_fw.bedGraph


In [5]:
# ЯЧЕЙКА 5: Загрузка bedGraph-файлов Borzoi (fw) в оперативную память
from tqdm import tqdm

bedgraph_files = list(set(experiment_mapping.values()))
bedgraph_data = {}

for fname in tqdm(bedgraph_files, desc="Loading bedGraph files"):
    full_path = os.path.join(BORZOI_BEDGRAPH_DIR, fname)
    if not os.path.exists(full_path):
        print("Файл не найден:", fname)
        continue
    df_tmp = pd.read_csv(full_path, sep="\t", header=None, names=["chrom","start","end","value"])
    bedgraph_data[fname] = df_tmp
    
    print(f"\nЗагружен {fname}, shape={df_tmp.shape}")
    display(df_tmp.head(3))
    print("value stats: min=%.3f max=%.3f mean=%.3f" % 
          (df_tmp["value"].min(), df_tmp["value"].max(), df_tmp["value"].mean()))


Обработка файлов:   0%|                                                                                                                                          | 0/10 [00:00<?, ?it/s]


Загружен файл borzoi_rnaseq_fold4_ENCSR357BYU+_fw.bedGraph, shape=(10799537, 4)


,chrom,start,end,value
0,chr10,72890684,72890716,0.538787
1,chr10,72890716,72890748,0.216714
2,chr10,72890748,72890780,0.051275


Обработка файлов:  10%|█████████████                                                                                                                     | 1/10 [00:03<00:32,  3.59s/it]

value: min=0.000 max=4004.835 mean=1.180

Загружен файл borzoi_rnaseq_fold4_ENCSR892LBU-_fw.bedGraph, shape=(10799537, 4)


...

Обработка файлов:  90%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████             | 9/10 [00:58<00:06,  6.40s/it]

value: min=0.000 max=4382.543 mean=1.627

Загружен файл borzoi_rnaseq_fold4_ENCSR763OMY+_fw.bedGraph, shape=(10799537, 4)


,chrom,start,end,value
0,chr10,72890684,72890716,0.141283
1,chr10,72890716,72890748,0.042791
2,chr10,72890748,72890780,0.006480


Обработка файлов: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:05<00:00,  6.54s/it]

value: min=0.000 max=9742.334 mean=2.038


In [6]:
# ЯЧЕЙКА 6: Логарифмирование coverage 

PSEUDOCOUNT = 0.01

bedgraph_data_log = {}
for fname, df in bedgraph_data.items():
    df_log = df.copy()
    # логарифмируем: ln(value + 0.01)
    df_log["value"] = np.log(df_log["value"] + PSEUDOCOUNT)
    bedgraph_data_log[fname] = df_log
    
    print(f"\nЛогарифмирован файл {fname}")
    print("value(min/max/mean) =", 
          df_log["value"].min(), 
          df_log["value"].max(), 
          df_log["value"].mean())



Логарифмирован файл borzoi_rnaseq_fold4_ENCSR357BYU+_fw.bedGraph
value(min/max/mean) = -4.604050246033248 8.29526008641543 -2.782012479944866

Логарифмирован файл borzoi_rnaseq_fold4_ENCSR892LBU-_fw.bedGraph
value(min/max/mean) = -4.603217062228037 8.263734575941122 -2.3667830958282927

Логарифмирован файл borzoi_rnaseq_fold4_ENCSR045GTF-_fw.bedGraph
value(min/max/mean) = -4.603812122672792 8.33599800939897 -2.4426745504733556

Логарифмирован файл borzoi_rnaseq_fold4_ENCSR357BYU-_fw.bedGraph
value(min/max/mean) = -4.603574056001573 8.070751273337727 -2.7398935592542215

Логарифмирован файл borzoi_rnaseq_fold4_ENCSR763OMY-_fw.bedGraph
value(min/max/mean) = -4.604979469312392 8.779723568889084 -3.0078493281054324

Логарифмирован файл borzoi_rnaseq_fold4_ENCSR892LBU+_fw.bedGraph
value(min/max/mean) = -4.604097877510423 8.636466477209188 -2.470956381734402

Логарифмирован файл borzoi_rnaseq_fold4_ENCSR045GTF+_fw.bedGraph
value(min/max/mean) = -4.604383714027222 8.765460701302153 -2.523270

In [7]:
# ЯЧЕЙКА 7: Подсчёт transcript-level покрытий Борзого, с учётом хромосомы [start..end]

preds_borzoi = []

for _, row in tqdm(labels_transcripts_df.iterrows(), total=labels_transcripts_df.shape[0], desc="Borzoi coverage"):
    t_id   = row["Transcript stable ID"]
    did    = row["description_id"]  
    ideal  = row["preds"]  # лог "идеальных меток" (RMT vs ...)

    # Ищем координаты [chrom, start, end] в labels_clear_df
    sub = labels_clear_df[labels_clear_df["Transcript stable ID"] == t_id]
    if sub.empty:
        preds_borzoi.append([t_id, did, np.nan, ideal])
        continue
    
    # Возьмём первую строку
    chrom  = sub.iloc[0]["chrom"]
    start_ = sub.iloc[0]["start"]
    end_   = sub.iloc[0]["end"]
    
    # Находим имя bedGraph
    bedfname = experiment_mapping.get(did, None)
    if bedfname is None:
        preds_borzoi.append([t_id, did, np.nan, ideal])
        continue
    
    # Достаём лог-файл
    df_cov = bedgraph_data_log.get(bedfname, None)
    if df_cov is None:
        preds_borzoi.append([t_id, did, np.nan, ideal])
        continue
    
    # Фильтруем: та же хромосома и [start..end]
    region = df_cov[
        (df_cov["chrom"] == chrom) &
        (df_cov["start"] >= start_) &
        (df_cov["end"]   <= end_)
    ]
    
    if region.empty:
        # если пусто: берём 0.0 (что в лог.шкале ~ ln(0.01)= -4.6)
        preds_borzoi.append([t_id, did, 0.0, ideal])
    else:
        mean_logcov = region["value"].mean()
        preds_borzoi.append([t_id, did, mean_logcov, ideal])

preds_borzoi_df = pd.DataFrame(preds_borzoi, columns=["Transcript stable ID","description_id","pred_borzoi","pred_ideal"])
print("preds_borzoi_df shape:", preds_borzoi_df.shape)
display(preds_borzoi_df.head(10))


Borzoi coverage: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22095/22095 [3:20:05<00:00,  1.84it/s]


In [9]:
# ЯЧЕЙКА 8: Объединяем c RMT (2/4/8seg)

df_merged = preds_borzoi_df.copy()

# 2seg
temp = preds_2seg_df[["Transcript stable ID","description_id","preds"]].rename(columns={"preds":"pred_rmt_2"})
df_merged = pd.merge(df_merged, temp, on=["Transcript stable ID","description_id"], how="left")

# 4seg
temp = preds_4seg_df[["Transcript stable ID","description_id","preds"]].rename(columns={"preds":"pred_rmt_4"})
df_merged = pd.merge(df_merged, temp, on=["Transcript stable ID","description_id"], how="left")

# 8seg
temp = preds_8seg_df[["Transcript stable ID","description_id","preds"]].rename(columns={"preds":"pred_rmt_8"})
df_merged = pd.merge(df_merged, temp, on=["Transcript stable ID","description_id"], how="left")

print("df_merged:", df_merged.shape)
display(df_merged.head())

# Статистика и корреляция (транскрипт-уровень)
for col in ["pred_borzoi","pred_ideal","pred_rmt_2","pred_rmt_4","pred_rmt_8"]:
    if col in df_merged.columns:
        print(f"{col} min={df_merged[col].min()} max={df_merged[col].max()} mean={df_merged[col].mean()}")

corr_cols = ["pred_borzoi","pred_ideal","pred_rmt_2","pred_rmt_4","pred_rmt_8"]
df_corr = df_merged[corr_cols].corr(method="pearson")
print("\nTranscript-level корреляции (Pearson):")
display(df_corr)


df_merged final shape: (22095, 7)


,Transcript stable ID,description_id,pred_borzoi,pred_ideal,pred_rmt_2,pred_rmt_4,pred_rmt_8
0,ENST00000001008,ENCFF123KIW,4.712871,4.037295,0.570057,2.727709,3.736615
1,ENST00000002596,ENCFF123KIWr,1.099219,4.281860,-2.881656,0.732307,1.674160
2,ENST00000002829,ENCFF123KIW,3.475321,5.388129,1.192969,2.426938,0.736134
3,ENST00000009180,ENCFF123KIW,6.688652,7.723541,2.320119,4.060136,4.146430
4,ENST00000009589,ENCFF123KIWr,5.853962,8.866660,4.619201,4.862711,5.075033


pred_borzoi min=-4.578855155245346 max=8.14484626177008 mean=0.19121216321998807
pred_ideal min=-4.605170200000001 max=11.856355566803279 mean=-0.4990853600718094
pred_rmt_2 min=-3.8326391907692305 max=6.641538864634146 mean=-1.7986488347158898
pred_rmt_4 min=-4.2842083040022745 max=7.001022550393701 mean=-0.715264122164497
pred_rmt_8 min=-4.180466452112676 max=6.389540544094489 mean=-0.6615956298938233

Корреляции (Pearson):


,pred_borzoi,pred_ideal,pred_rmt_2,pred_rmt_4,pred_rmt_8
pred_borzoi,1.000000,0.784860,0.657547,0.779853,0.782714
pred_ideal,0.784860,1.000000,0.666830,0.781221,0.778404
pred_rmt_2,0.657547,0.666830,1.000000,0.818327,0.773918
pred_rmt_4,0.779853,0.781221,0.818327,1.000000,0.960751
pred_rmt_8,0.782714,0.778404,0.773918,0.960751,1.000000


In [17]:
# ЯЧЕЙКА 9: Загрузка t_g_df -> хранит сопоставление transcript->gene

t_g_file = os.path.join(RMT_INFER_DIR, "t_g.tsv")
t_g_df = pd.read_csv(t_g_file, sep="\t")

print("Первые 5 строк t_g_df:")
display(t_g_df.head())

print("\nИнформация о t_g_df:")
t_g_df.info()


Первые 5 строк t_g_df:


,Unnamed: 0,chrom,name,strand,strand.1,split,Transcript stable ID,start,end
0,29191,chr3,ENST00000002829,1,ENSG00000001617,valid,ENST00000002829,50140727,50189075
1,29192,chr12,ENST00000358495,-1,ENSG00000002016,valid,ENST00000358495,911735,964290
2,29193,chr4,ENST00000226299,1,ENSG00000002549,valid,ENST00000226299,17562601,17607970
3,29194,chr4,ENST00000002596,-1,ENSG00000002587,valid,ENST00000002596,11393149,11443490
4,29195,chr2,ENST00000264276,-1,ENSG00000003393,valid,ENST00000264276,201700266,201795529



Информация о t_g_df:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4419 entries, 0 to 4418
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Unnamed: 0            4419 non-null   int64 
 1   chrom                 4419 non-null   object
 2   name                  4419 non-null   object
 3   strand                4419 non-null   int64 
 4   strand.1              4419 non-null   object
 5   split                 4419 non-null   object
 6   Transcript stable ID  4419 non-null   object
 7   start                 4419 non-null   int64 
 8   end                   4419 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 310.8+ KB


In [22]:
# =================== ЯЧЕЙКА 9 ===================
# Загрузка TPM-таблиц ENCODE (для каждого эксперимента) и сохранение в словарь.
# (Предполагается, что df_merged у нас уже есть из предыдущих ячеек)

RMT_INFER_DIR = "/mnt/10tb/home/patsukevichan/borZoi/borzoi/examples/rmt_infer/"

encode_gene_tpm_files = {
    "ENCSR071DYD": "ENCFF258PUQ ENCSR071DYD.tsv",
    "ENCSR763OMY": "ENCFF993FDD ENCSR763OMY.tsv",
    "ENCSR357BYU": "ENCFF296APA ENCSR357BYU.tsv",
    "ENCSR892LBU": "ENCFF653OLW ENCSR892LBU.tsv",
    "ENCSR045GTF": "ENCFF769IPE ENCSR045GTF.tsv"
}

encode_gene_tpm_data = {}

for experiment_id, fname in encode_gene_tpm_files.items():
    fpath = os.path.join(RMT_INFER_DIR, fname)
    if not os.path.exists(fpath):
        print(f"[WARN] Файл TPM для эксперимента {experiment_id} не найден: {fname}")
        continue

    df_tpm = pd.read_csv(fpath, sep="\t")
    # Предположим, что нас интересуют столбцы: 'gene_id', 'transcript_id(s)', 'TPM'
    # (Остальные: 'expected_count', 'FPKM', 'posterior_mean_count' и т.д. — не трогаем)
    # Убедитесь, что реально есть столбцы 'gene_id', 'transcript_id(s)', 'TPM'
    if not {"gene_id","transcript_id(s)","TPM"}.issubset(df_tpm.columns):
        print(f"[WARN] Отсутствуют нужные колонки в {fname}. Пропуск.")
        continue

    print(f"\n[OK] Загрузили файл: {fname} (experiment_id={experiment_id}), shape={df_tpm.shape}")
    display(df_tpm.head(3))

    encode_gene_tpm_data[experiment_id] = df_tpm



[OK] Загрузили файл: ENCFF258PUQ ENCSR071DYD.tsv (experiment_id=ENCSR071DYD), shape=(59526, 17)


,gene_id,transcript_id(s),length,effective_length,expected_count,TPM,FPKM,posterior_mean_count,posterior_standard_deviation_of_count,pme_TPM,pme_FPKM,TPM_ci_lower_bound,TPM_ci_upper_bound,TPM_coefficient_of_quartile_variation,FPKM_ci_lower_bound,FPKM_ci_upper_bound,FPKM_coefficient_of_quartile_variation
0,10904,10904,93.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12954,12954,94.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12956,12956,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0



[OK] Загрузили файл: ENCFF993FDD ENCSR763OMY.tsv (experiment_id=ENCSR763OMY), shape=(59526, 17)


,gene_id,transcript_id(s),length,effective_length,expected_count,TPM,FPKM,posterior_mean_count,posterior_standard_deviation_of_count,pme_TPM,pme_FPKM,TPM_ci_lower_bound,TPM_ci_upper_bound,TPM_coefficient_of_quartile_variation,FPKM_ci_lower_bound,FPKM_ci_upper_bound,FPKM_coefficient_of_quartile_variation
0,10904,10904,93.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12954,12954,94.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12956,12956,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0



[OK] Загрузили файл: ENCFF296APA ENCSR357BYU.tsv (experiment_id=ENCSR357BYU), shape=(59526, 17)


,gene_id,transcript_id(s),length,effective_length,expected_count,TPM,FPKM,posterior_mean_count,posterior_standard_deviation_of_count,pme_TPM,pme_FPKM,TPM_ci_lower_bound,TPM_ci_upper_bound,TPM_coefficient_of_quartile_variation,FPKM_ci_lower_bound,FPKM_ci_upper_bound,FPKM_coefficient_of_quartile_variation
0,10904,10904,93.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12954,12954,94.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12956,12956,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0



[OK] Загрузили файл: ENCFF653OLW ENCSR892LBU.tsv (experiment_id=ENCSR892LBU), shape=(59526, 17)


,gene_id,transcript_id(s),length,effective_length,expected_count,TPM,FPKM,posterior_mean_count,posterior_standard_deviation_of_count,pme_TPM,pme_FPKM,TPM_ci_lower_bound,TPM_ci_upper_bound,TPM_coefficient_of_quartile_variation,FPKM_ci_lower_bound,FPKM_ci_upper_bound,FPKM_coefficient_of_quartile_variation
0,10904,10904,93.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12954,12954,94.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12956,12956,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0



[OK] Загрузили файл: ENCFF769IPE ENCSR045GTF.tsv (experiment_id=ENCSR045GTF), shape=(59526, 17)


,gene_id,transcript_id(s),length,effective_length,expected_count,TPM,FPKM,posterior_mean_count,posterior_standard_deviation_of_count,pme_TPM,pme_FPKM,TPM_ci_lower_bound,TPM_ci_upper_bound,TPM_coefficient_of_quartile_variation,FPKM_ci_lower_bound,FPKM_ci_upper_bound,FPKM_coefficient_of_quartile_variation
0,10904,10904,93.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12954,12954,94.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12956,12956,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
# =================== ЯЧЕЙКА 10 ===================
# Подготавливаем функции strip_enst_version(...) и parse_transcript_ids(...),
# чтобы корректно «обрезать» версии у транскриптов (если ".2" и т.п.).

def strip_enst_version(enst_str):
    """
    "ENST00000001008.2" -> "ENST00000001008"
    Если нет точки, возвращаем строку без изменений.
    """
    if '.' in enst_str:
        return enst_str.split('.', 1)[0]
    return enst_str

def parse_transcript_ids(text):
    """
    Разбиваем строку типа: "ENST00000001008.2,ENST00000002829.3" на список
    ["ENST00000001008","ENST00000002829"] (обрезая версии).
    """
    if pd.isna(text):
        return []
    raw_ids = text.split(',')
    cleaned_ids = []
    for rid in raw_ids:
        rid_stripped = rid.strip()
        rid_stripped = strip_enst_version(rid_stripped)
        cleaned_ids.append(rid_stripped)
    return cleaned_ids


In [24]:
# =================== ЯЧЕЙКА 11 ===================
# Функция для добавления TPM к df_merged, используя transcript_id(s).
# При этом (a) "explode" каждую строчку df_tpm по списку transcript_ids
# (b) мержим с df_merged по 'Transcript stable ID'.

def add_tpm_by_transcript(df_merged_sub, df_tpm):
    """
    df_merged_sub : DataFrame со столбцом 'Transcript stable ID' (ENST…),
                    где версия уже, желательно, убрана (или не было изначально).

    df_tpm        : DataFrame с 'gene_id', 'transcript_id(s)', 'TPM'.
                    Примерная структура ENCODE-файлов.

    Возвращаем df_out, который содержит все столбцы из df_merged_sub +
    ['gene_id','TPM'] из df_tpm.
    """
    df_tmp = df_tpm.copy()

    # 1) создаём столбец list_of_transcripts, затем "explode"
    df_tmp["list_of_transcripts"] = df_tmp["transcript_id(s)"].apply(parse_transcript_ids)
    df_exploded = df_tmp.explode("list_of_transcripts", ignore_index=True)

    # 2) Переименуем "list_of_transcripts" -> "Transcript stable ID" для merge
    df_exploded.rename(columns={"list_of_transcripts":"Transcript stable ID"}, inplace=True)

    # 3) Теперь у df_exploded есть колонки "gene_id","TPM","Transcript stable ID" и т.д.
    # Мержим left join
    df_out = pd.merge(
        df_merged_sub,
        df_exploded[["gene_id","TPM","Transcript stable ID"]],
        on="Transcript stable ID",
        how="left"
    )
    return df_out


In [25]:
# =================== ЯЧЕЙКА 12 ===================
# Итоговая ячейка: Для каждого эксперимента (ENCSR*), 
# фильтруем df_merged, добавляем TPM, считаем корреляцию.

def desc_to_experiment(did):
    """
    Примерная функция: 'ENCFF123KIW' -> 'ENCSR892LBU'
    Убедитесь, что строки match-ятся корректно для всех description_id!
    """
    if   "KIW" in did: return "ENCSR892LBU"
    elif "MDF" in did: return "ENCSR357BYU"
    elif "XOK" in did: return "ENCSR763OMY"
    elif "TTC" in did: return "ENCSR071DYD"
    elif "BWW" in did: return "ENCSR045GTF"
    return None

# Убедимся, что в df_merged есть столбец 'experiment'
df_merged["experiment"] = df_merged["description_id"].apply(desc_to_experiment)

# Для удобства выведем experiment'ы, чтобы увидеть, всё ли замапилось
print("Unique experiment values in df_merged:", df_merged["experiment"].unique())

# Столбцы, по которым хотим коррелять
analysis_cols = ["pred_borzoi","pred_ideal","pred_rmt_2","pred_rmt_4","pred_rmt_8","TPM"]

for experiment_id, df_tpm in encode_gene_tpm_data.items():
    # 1) Выбираем строки df_merged по experiment_id
    df_merged_sub = df_merged[df_merged["experiment"] == experiment_id].copy()
    if df_merged_sub.empty:
        print(f"[{experiment_id}] Нет строк в df_merged => skip.\n")
        continue

    # 2) Убедимся, что у 'Transcript stable ID' нет версий (вдруг у вас есть).
    #    Если есть - обрежьте:
    df_merged_sub["Transcript stable ID"] = df_merged_sub["Transcript stable ID"].apply(strip_enst_version)

    # 3) Добавляем TPM
    df_with_tpm = add_tpm_by_transcript(df_merged_sub, df_tpm)

    # 4) Считаем transcript-level корреляцию
    # Отбираем только нужные столбцы, убираем NaN
    sub_for_corr = df_with_tpm[analysis_cols].dropna()
    n_rows = len(sub_for_corr)
    print(f"\n=== Эксперимент: {experiment_id} ===")
    print(f"Размер после merge (transcript-level) = {len(df_with_tpm)} строк, "
          f"после dropna = {n_rows} строк.")

    if n_rows < 5:
        print(f"Слишком мало строк для корреляции => skip.")
        continue

    cor_transcript = sub_for_corr.corr(method="pearson")
    print("[Transcript-level] корреляция:")
    display(cor_transcript)

    # 5) Считаем gene-level (пример: усредняем все транскрипты одного гена)
    #    У нас есть столбец 'gene_id' из df_with_tpm
    df_gene = df_with_tpm.dropna(subset=["gene_id"])[
        ["gene_id"] + analysis_cols
    ].groupby("gene_id", dropna=False).mean().reset_index()

    sub_gene_for_corr = df_gene[analysis_cols].dropna()
    n_gene = len(sub_gene_for_corr)
    print(f"Генов после группировки: {len(df_gene)}, после dropna={n_gene}.")
    if n_gene < 5:
        print("Слишком мало генов для корреляции => skip.\n")
        continue

    cor_gene = sub_gene_for_corr.corr(method="pearson")
    print("[Gene-level] корреляция:")
    display(cor_gene)
    print("-------------------------------------------------\n")


Unique experiment values in df_merged: ['ENCSR892LBU' 'ENCSR357BYU' 'ENCSR763OMY' 'ENCSR071DYD' 'ENCSR045GTF']

=== Эксперимент: ENCSR071DYD ===
Размер после merge (transcript-level) = 4419 строк, после dropna = 3639 строк.
[Transcript-level] корреляция:


,pred_borzoi,pred_ideal,pred_rmt_2,pred_rmt_4,pred_rmt_8,TPM
pred_borzoi,1.000000,0.773383,0.643206,0.774777,0.779495,0.068931
pred_ideal,0.773383,1.000000,0.662825,0.781934,0.786345,0.128468
pred_rmt_2,0.643206,0.662825,1.000000,0.811457,0.761510,0.054425
pred_rmt_4,0.774777,0.781934,0.811457,1.000000,0.963923,0.037674
pred_rmt_8,0.779495,0.786345,0.761510,0.963923,1.000000,0.033981
TPM,0.068931,0.128468,0.054425,0.037674,0.033981,1.000000


Генов после группировки: 3624, после dropna=3624.
[Gene-level] корреляция:


,pred_borzoi,pred_ideal,pred_rmt_2,pred_rmt_4,pred_rmt_8,TPM
pred_borzoi,1.000000,0.774338,0.644724,0.775552,0.780243,0.068977
pred_ideal,0.774338,1.000000,0.663936,0.783257,0.787328,0.128575
pred_rmt_2,0.644724,0.663936,1.000000,0.811978,0.762263,0.054503
pred_rmt_4,0.775552,0.783257,0.811978,1.000000,0.963983,0.037612
pred_rmt_8,0.780243,0.787328,0.762263,0.963983,1.000000,0.033887
TPM,0.068977,0.128575,0.054503,0.037612,0.033887,1.000000


-------------------------------------------------


=== Эксперимент: ENCSR763OMY ===
Размер после merge (transcript-level) = 4419 строк, после dropna = 3639 строк.
[Transcript-level] корреляция:


...

-------------------------------------------------


=== Эксперимент: ENCSR045GTF ===
Размер после merge (transcript-level) = 4419 строк, после dropna = 3639 строк.
[Transcript-level] корреляция:


,pred_borzoi,pred_ideal,pred_rmt_2,pred_rmt_4,pred_rmt_8,TPM
pred_borzoi,1.000000,0.785007,0.652311,0.780401,0.783012,0.120785
pred_ideal,0.785007,1.000000,0.657162,0.775700,0.775005,0.236554
pred_rmt_2,0.652311,0.657162,1.000000,0.811937,0.764098,0.152274
pred_rmt_4,0.780401,0.775700,0.811937,1.000000,0.958364,0.132003
pred_rmt_8,0.783012,0.775005,0.764098,0.958364,1.000000,0.127538
TPM,0.120785,0.236554,0.152274,0.132003,0.127538,1.000000


Генов после группировки: 3624, после dropna=3624.
[Gene-level] корреляция:


,pred_borzoi,pred_ideal,pred_rmt_2,pred_rmt_4,pred_rmt_8,TPM
pred_borzoi,1.000000,0.786501,0.654027,0.781345,0.783919,0.120882
pred_ideal,0.786501,1.000000,0.658835,0.777319,0.776264,0.235998
pred_rmt_2,0.654027,0.658835,1.000000,0.812432,0.764967,0.151830
pred_rmt_4,0.781345,0.777319,0.812432,1.000000,0.958444,0.131481
pred_rmt_8,0.783919,0.776264,0.764967,0.958444,1.000000,0.126887
TPM,0.120882,0.235998,0.151830,0.131481,0.126887,1.000000


-------------------------------------------------

